<a href="https://colab.research.google.com/github/ddah0329/SeoulPublicBike-Analysis/blob/main/rental_analysis_by_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive/')

# 접근하려는 폴더 설정
FOLDERNAME = 'SeoulPublicBike-Analysis'
TARGET_PATH = f'/content/drive/MyDrive/{FOLDERNAME}'

# 접근하려는 폴더로 디렉토리 이동
%cd $TARGET_PATH

# sys.path에서 다른 드라이브 경로 제거 (불필요한 접근 방지)
import sys
original_paths = sys.path.copy()
sys.path = [p for p in sys.path if p.startswith('/usr/') or p.startswith('/env/') or p == TARGET_PATH]

print("📁 현재 폴더:", TARGET_PATH)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/SeoulPublicBike-Analysis
📁 현재 폴더: /content/drive/MyDrive/SeoulPublicBike-Analysis


In [ ]:
# 필요한 모듈 import하기
import os
import pandas as pd

# 1. 데이터 불러오기
file_name = 'raw_csv/bicycle_rental_ by_time .csv'
df = pd.read_csv(file_name, encoding='cp949')

In [ ]:
# 2. 열 이름 영어로 변경
column_name_mapping = {
    '기준_날짜': 'RENT_DATE',
    '기준_시간대': 'RENT_TIME',
    '시작_대여소명': 'RENT_STATION_NAME',
    '종료_대여소명': 'RETURN_STATION_NAME',
    '전체_건수': 'RENT_COUNT',
    '전체_이용_분': 'RENT_DURATION_MIN',
    '전체_이용_거리': 'RENT_DISTANCE_METER'
}
df.rename(columns = column_name_mapping, inplace=True)


# 3. 시간 파싱 및 시간대 그룹화
df['RENT_DATE'] = pd.to_datetime(df['RENT_DATE'], format='%Y%m%d')
df['hour'] = df['RENT_TIME'].astype(str).str.zfill(4).str[:2].astype(int)

def group_time(hour):
    if 6 <= hour < 9:
        return '06~09'
    elif 9 <= hour < 12:
        return '09~12'
    elif 12 <= hour < 15:
        return '12~15'
    elif 15 <= hour < 18:
        return '15~18'
    elif 18 <= hour < 21:
        return '18~21'
    else:
        return '기타'

df['time_slot'] = df['hour'].apply(group_time)


# 4. 피벗 테이블 생성
pivot_table = pd.pivot_table(
    df,
    values='RENT_COUNT',
    index='RENT_STATION_NAME',
    columns='time_slot',
    aggfunc='sum',
    fill_value=0
)

# 5. 결과 출력
print("모든 대여소의 시간대별 대여 건수 요약표:")
print(pivot_table)

모든 대여소의 시간대별 대여 건수 요약표:
time_slot          06~09  09~12  12~15  15~18  18~21  기타
RENT_STATION_NAME                                       
가락1동_039_1            50     24     43     61     92  46
가락1동_040_1            31      3      8     18      6  10
가락1동_048_1            34     10      6     17     40  13
가락1동_048_2             8      8      2      0      6   0
가락1동_048_3            14     15     37     27     53  22
...                  ...    ...    ...    ...    ...  ..
흑석동_024_1             18     10      3      3      7   9
흑석동_033_1              0      1      1      0      2   2
흑석동_038_1              7      7      4      2      0   2
흑석동_038_2              2      0      2      0      0   4
흑석동_058_1             10      6      8      6      4   2

[2718 rows x 6 columns]


In [ ]:
# 6. 저장 및 다운로드
output_file = "processed_csv/rent_summary_by_station_timeslot.csv"
pivot_table.to_csv(output_file, encoding='cp949')